In [ ]:
import torch
import torch.utils.dlpack
import open3d as o3d
import numpy as np

## Tensor creation

In [ ]:
# From list
a = o3d.Tensor([0, 1, 2])
print(a)

# Dtype inferred from list
a_float = o3d.Tensor([0.0, 1.0, 2.0])
print(a_float)

# From Numpy
a = o3d.Tensor(np.array([0, 1, 2]))
print(a)

# Specify dtype
a = o3d.Tensor(np.array([0, 1, 2]), dtype=o3d.Dtype.Float64)
print(a)

# Specify device
# Specify dtype
a = o3d.Tensor(np.array([0, 1, 2]), device=o3d.Device("CUDA:0"))
print(a)

## Properties of a tensor

In [ ]:
vals = np.array((range(24))).reshape(2, 3, 4)
a = o3d.Tensor(np.array(vals), 
               dtype=o3d.Dtype.Float64,
               device=o3d.Device("CUDA:0"))
print(f"a.shape: {a.shape}")
print(f"a.strides: {a.strides}")
print(f"a.dtype: {a.dtype}")
print(f"a.device: {a.device}")
print(f"a.ndim: {a.ndim}")

## Copy & device transfer

In [ ]:
# Host -> Device
a_cpu = o3d.Tensor([0, 1, 2])
a_gpu = a_cpu.cuda(0)
print(a_gpu)

# Device -> Device
a_gpu = o3d.Tensor([0, 1, 2], device=o3d.Device("CUDA:0"))
a_cpu = a_gpu.cpu()
print(a_cpu)

# Device -> Host
a_gpu_0 = o3d.Tensor([0, 1, 2], device=o3d.Device("CUDA:0"))
a_gpu_1 = a_gpu_0.cuda(1)
print(a_gpu_1)

## Type casting

In [ ]:
# E.g. float -> int
a = o3d.Tensor([0.1, 1.5, 2.7])
b = a.to(o3d.Dtype.Int32)
print(a)
print(b)

## Numpy I/O with direct memory map

In [ ]:
# From numpy
np_a = np.ones((5,), dtype=np.int32)
o3_a = o3d.Tensor.from_numpy(np_a)
print(f"np_a: {np_a}")
print(f"o3_a: {o3_a}")
print("")

# Changes to numpy array reflects on open3d Tensor and vice versa
np_a[0] = 100
o3_a[1] = 200
print(f"np_a: {np_a}")
print(f"o3_a: {o3_a}")

In [ ]:
# To numpy
o3_a = o3d.Tensor([1, 1, 1, 1, 1], dtype=o3d.Dtype.Int32)
np_a = o3_a.numpy()

# Changes to numpy array reflects on open3d Tensor and vice versa
np_a[0] = 100
o3_a[1] = 200
print(f"np_a: {np_a}")
print(f"o3_a: {o3_a}")

# For CUDA Tensor, call cpu() before calling numpy()
o3_a = o3d.Tensor([1, 1, 1, 1, 1], device=o3d.Device("CUDA:0"))
print(f"\no3_a.cpu().numpy(): {o3_a.cpu().numpy()}")

## PyTorch I/O with DLPack memory map

In [ ]:
# From PyTorch
th_a = torch.ones((5,)).cuda(0)
o3_a = o3d.Tensor.from_dlpack(torch.utils.dlpack.to_dlpack(th_a))
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")
print("")

# Changes to PyTorch array reflects on open3d Tensor and vice versa
th_a[0] = 100
o3_a[1] = 200
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")

In [ ]:
# To PyTorch
o3_a = o3d.Tensor([1, 1, 1, 1, 1], device=o3d.Device("CUDA:0"))
th_a = torch.utils.dlpack.from_dlpack(o3_a.to_dlpack())
o3_a = o3d.Tensor.from_dlpack(torch.utils.dlpack.to_dlpack(th_a))
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")
print("")

# Changes to PyTorch array reflects on open3d Tensor and vice versa
th_a[0] = 100
o3_a[1] = 200
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")

## Binary element-wise operation: add, sub, mul, div, ...

In [ ]:
a = o3d.Tensor([1, 1, 1], dtype=o3d.Dtype.Float32)
b = o3d.Tensor([2, 2, 2], dtype=o3d.Dtype.Float32)
print(a + b)
print(a - b)
print(a * b)
print(a / b)
print("")

# Automated broadcasting
a = o3d.Tensor(np.ones((2, 3)), dtype=o3d.Dtype.Float32)
b = o3d.Tensor(np.ones((3,)), dtype=o3d.Dtype.Float32)
print(a + b)

## Unary element-wise operation: sqrt, sin, cos, ...

In [ ]:
a = o3d.Tensor([4, 9, 16], dtype=o3d.Dtype.Float32)
print(a.sqrt())
print(a.sin())
print(a.cos())

## Reduction: sum, prod, min, max

In [ ]:
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3d.Tensor(vals)
print(a.sum())
print(a.prod())
print(a.min())
print(a.max())

In [ ]:
# With specified dimension
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3d.Tensor(vals)

print(a.sum(dim=(0)), "\n")
print(a.sum(dim=(0, 2)), "\n")
print(a.sum(dim=(0, 2), keepdim=True))

## Slicing, indexing, getitem (returns a view), and setitem

In [ ]:
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3d.Tensor(vals)

# Slicing __getitem__
print(a[1:], "\n")

# Indexing __getitem__
print(a[1, 2], "\n")

# Combined __getitem__
print(a[:-1, 0:3:2, 2])

In [ ]:
# Example __setitem__
print(a, "\n")
a[:, :, 2] = 100
print(a)

## Advanced indexing

In [ ]:
# Pybind not done yet, see C++ docs